In [ ]:
import pandas as pd
import feather
import os
import numpy as np

import plotly.express as px

import json
from pandas.io.json import json_normalize

In [ ]:
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
test_df = load_df('data/test_v2.csv')

In [ ]:
train_df = load_df('data/train_v2.csv')

In [ ]:
#train_df.select_dtypes('int64')
train_df[['visitId','visitNumber','visitStartTime']] =train_df[['visitId','visitNumber','visitStartTime']].astype('int32')
test_df[['visitId','visitNumber','visitStartTime']] =test_df[['visitId','visitNumber','visitStartTime']].astype('int32')

In [ ]:
train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype("float")

In [ ]:
#Date Column
train_df['date'] = pd.to_datetime(train_df['date'],format="%Y%m%d")
test_df['date'] = pd.to_datetime(test_df['date'],format="%Y%m%d")

In [ ]:
# Delete columns with single value
single_value_cols = [c for c in train_df.columns if train_df[c].nunique(dropna= False)==1]

In [ ]:
train_df.drop(single_value_cols,inplace=True,axis=1)
test_df.drop(single_value_cols,inplace=True,axis=1)

In [ ]:
set(train_df.columns).difference(set(test_df.columns))

In [ ]:
train_df.drop(['trafficSource.campaignCode'],axis = 1,inplace=True)

In [ ]:
test_df.drop('totals.transactionRevenue',axis = 1,inplace=True)

In [ ]:
train_df['totals.transactionRevenue'].fillna(0,inplace=True)

In [ ]:
train_df.to_csv("data/train_v3.csv")
test_df.to_csv("data/test_v3.csv")

In [ ]:
test_df['fullVisitorId'].nunique()